In [ ]:
import json
import math
from collections import Counter

import matplotlib.cm as cm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pywt
from IPython.display import clear_output, display
from qiskit import (Aer, ClassicalRegister, QuantumCircuit, QuantumRegister,
                    assemble, execute, transpile)
from qiskit.algorithms.optimizers import ADAM, COBYLA, GradientDescent
from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit
from qiskit.circuit.library import (CRXGate, CRZGate, CXGate, EfficientSU2,
                                    HGate, NLocal, PauliFeatureMap,
                                    RealAmplitudes, RXGate, RYGate, RZGate,
                                    TwoLocal, ZFeatureMap, ZZFeatureMap)
from qiskit.extensions import Initialize
from qiskit.primitives import Sampler
from qiskit.quantum_info import (Operator, SparsePauliOp, Statevector,
                                 partial_trace, random_statevector)
from qiskit.tools.visualization import (array_to_latex, plot_bloch_multivector,
                                        plot_histogram)
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import (
    VQC, NeuralNetworkClassifier)
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_textbook.tools import simon_oracle
from skimage import transform
from sklearn import datasets
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn import metrics

algorithm_globals.random_seed = 42

In [ ]:
def plot_silhouette(X, labels, n_cluster, metric='euclidean'):
    X = X[labels != -1]
    labels = labels[labels != -1]
    sil_score = silhouette_score(X, labels, metric=metric)
    sil_samples = silhouette_samples(X, labels, metric=metric)

    fig, ax = plt.subplots()
    fig.set_size_inches(18, 10)
    ax.set_xlim([-0.1,1])
    ax.set_ylim([0, len(X) + (n_cluster + 1)*10])

    y_lower = 10
    for i in range(n_cluster):
        ith_cluster_silhouette_values = sil_samples[labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_cluster)

        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        ax.text(-0.05, y_lower + (0.5 * size_cluster_i), str(i))
        y_lower = y_upper + 10


    ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")
    ax.axvline(x=sil_score, color="red", linestyle="--")

    plt.show()

    return sil_score

In [ ]:
TRAIN_SIZE = 100

dataset, classes = datasets.make_moons(n_samples=TRAIN_SIZE, random_state=10)
dataset = np.array(dataset)

plt.scatter(dataset[:,0], dataset[:,1], c=classes, cmap="bwr_r")

In [ ]:
zz_map = ZZFeatureMap(feature_dimension=2, reps=1, entanglement='linear', insert_barriers=True)
zz_kernel = QuantumKernel(feature_map=zz_map, quantum_instance=Aer.get_backend('statevector_simulator'))

matrix_train = zz_kernel.evaluate(x_vec=dataset) 
plt.imshow(matrix_train,  interpolation='nearest', origin='upper', cmap='Blues')

In [ ]:
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [ ]:
clustering = AgglomerativeClustering(affinity='precomputed', n_clusters=2,  linkage='average').fit(matrix_train)

labels = clustering.labels_

print(f"Silhouette: {silhouette_score(dataset, labels, metric = 'euclidean')}")
contingency_matrix = metrics.cluster.contingency_matrix(classes, labels)
plt.matshow(contingency_matrix, )
plt.xlabel("Predicted")
plt.ylabel("True")
print(f"Purity: {purity_score(classes, labels)}")

In [ ]:
clustering = AgglomerativeClustering(n_clusters=2).fit(dataset)

labels = clustering.labels_

print(f"Silhouette: {silhouette_score(dataset, labels, metric = 'euclidean')}")
contingency_matrix = metrics.cluster.contingency_matrix(classes, labels)
plt.matshow(contingency_matrix)
plt.xlabel("Predicted")
plt.ylabel("True")
print(f"Purity: {purity_score(classes, labels)}")